In [12]:
import pandas as pd

### Listen Events Table Old

In [13]:
df = pd.read_csv(
    "/home/lupusruber/music_analytics/data/data/events.csv", encoding="utf-8"
)

In [14]:
transform_string = (
    lambda x: "".join([("_" + i if i.isupper() else i) for i in x]).strip().lower()
)

In [15]:
df.columns = list(map(transform_string, df.columns.tolist()))
df = df.rename({"ts": "timestamp"}, axis=1)

In [16]:
df.head(1)

,_unnamed: 0,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,0,Ayo,Welcome Into My World,264.56772,1616086802000,9782,Logged In,free,2,Flagstaff,86004,AZ,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-111.373463,35.380939,9783,Parker,Haley,F,1616086800000


In [17]:
df.query("timestamp == 1616086802000")

,_unnamed: 0,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,0,Ayo,Welcome Into My World,264.56772,1616086802000,9782,Logged In,free,2,Flagstaff,86004,AZ,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-111.373463,35.380939,9783,Parker,Haley,F,1616086800000


In [18]:
print(
    df[["session_id", "user_id"]].drop_duplicates().shape,
    df[["session_id", "user_id"]].shape,
)

(14662, 2) (100000, 2)


In [19]:
new_df = df[["session_id", "user_id"]].groupby("session_id").aggregate(lambda x: x)

In [20]:
import numpy as np

In [21]:
new_df["user_id"] = new_df["user_id"].apply(
    lambda x: list(x) if isinstance(x, (list, np.ndarray)) else [x]
)

In [22]:
new_df["truth"] = new_df["user_id"].apply(lambda x: len(set(x)) == 1)

In [23]:
new_df["truth"].unique()

array([ True])

In [24]:
type(new_df["user_id"].values[0])

list

In [25]:
df.groupby("session_id").count().reset_index()

,session_id,_unnamed: 0,artist,song,duration,timestamp,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,10,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,12,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
3,28,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,31,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14657,55788,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
14658,55805,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
14659,55848,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
14660,55882,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [26]:
gdf = df.groupby("gender")["gender"].count() / len(df)

In [27]:
df.query("user_id==748 and item_in_session==30").sort_values(
    "timestamp", ascending=True
).head(10)

,_unnamed: 0,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
650,650,Irma Thomas,The Same Love That Made Me Laugh [LP Version],195.36934,1616087608000,747,Logged In,paid,30,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748,Ramirez,Kyra,F,1616086800000
80143,80143,Chris Vickery,Jokers and Thieves,191.71220,1616302962000,49237,Logged In,paid,30,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748,Ramirez,Kyra,F,1616086800000


In [28]:
result = (
    df.groupby("session_id")
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)
# result['count'].sum() == n_rows
result

,session_id,count
7220,37432,219
7958,38794,216
13931,51602,199
4659,30891,197
8117,39079,178
...,...,...
4605,30649,1
4637,30787,1
4635,30784,1
4634,30781,1


In [29]:
df.query("session_id == 37432")[
    ["artist", "song", "duration", "item_in_session", "user_id"]
]["user_id"].unique()

array([17692])

In [30]:
session_and_user = (
    df.groupby(["session_id", "user_id"])
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)
# df.groupby(['sessionId', 'userId']).size().reset_index(name='count')['count].sum() == n_rows
session_and_user

,session_id,user_id,count
7220,37432,17692,219
7958,38794,17940,216
13931,51602,19676,199
4659,30891,16304,197
8117,39079,17780,178
...,...,...,...
4605,30649,1903,1
4637,30787,10466,1
4635,30784,13727,1
4634,30781,17442,1


In [31]:
session_and_user.groupby("user_id").size().reset_index(name="count").sort_values(
    "count", ascending=False
)

,user_id,count
5726,13473,17
3371,8010,16
121,288,15
3119,7357,13
3783,8943,12
...,...,...
8439,19986,1
8425,19958,1
8426,19959,1
8428,19964,1


In [32]:
session_and_user.sort_values("user_id")

,session_id,user_id,count
4132,28898,3,3
10935,44377,8,1
6093,35076,8,3
0,9,10,1
10626,43735,11,5
...,...,...,...
14085,52141,19996,1
13523,50407,19997,5
2870,19996,19997,1
7136,37261,20001,37


In [33]:
df["user_id"].unique().shape[0] == df["session_id"].unique().shape[0]

False

In [34]:
df.query("user_id == 19997").groupby("session_id").aggregate(lambda x: x)[
    "item_in_session"
]

session_id
19996                  0
50407    [0, 1, 3, 5, 6]
Name: item_in_session, dtype: object

### Auth Events Table

In [35]:
import pandas as pd

auth_events = pd.read_csv("/home/lupusruber/music_analytics/data/auth_events.csv")
print(auth_events.head(1)["userAgent"].values[0])
auth_events["ts"] = pd.to_datetime(auth_events["ts"], unit="ms")
auth_events["registration"] = pd.to_datetime(auth_events["registration"], unit="ms")

Mozilla/5.0 (X11; Linux x86_64; rv:31.0) Gecko/20100101 Firefox/31.0


In [36]:
mask = auth_events.query("userId == 75")[["ts"]].idxmin()
auth_events.loc[mask]

,ts,sessionId,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration,success
60,2021-01-23 12:42:01,814,free,5,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75.0,Buck,Oscar,M,2021-01-01,True


In [37]:
auth_events.groupby("userId").aggregate(
    {
        "ts": ["min", "max"],
    }
).reset_index()

userId                  ts                    
                          min                 max
0     2.0 2021-01-06 18:10:32 2021-01-06 18:10:32
1     3.0 2021-04-01 11:41:41 2021-10-27 17:51:22
2     4.0 2021-06-25 13:16:20 2021-07-16 11:47:04
3     5.0 2021-01-13 18:03:17 2021-10-20 16:35:58
4     6.0 2021-05-10 04:31:22 2021-11-27 12:25:43
..    ...                 ...                 ...
84   97.0 2021-03-17 03:44:39 2021-11-12 15:51:50
85   98.0 2021-01-14 18:41:11 2021-07-23 15:16:10
86   99.0 2021-01-31 09:47:32 2021-11-26 12:02:24
87  100.0 2021-11-14 11:26:18 2021-11-14 11:26:18
88  101.0 2021-07-13 18:27:42 2021-07-13 18:27:42

[89 rows x 3 columns]

### Page View Events Table

In [38]:
page_view = pd.read_csv("/home/lupusruber/music_analytics/data/page_view_events.csv")
page_view.drop(["userAgent"], inplace=True, axis=1)
page_view["ts"] = pd.to_datetime(page_view["ts"], unit="ms")
page_view["registration"] = pd.to_datetime(page_view["registration"], unit="ms")
page_view.sample(3)[["ts", "registration"]]

,ts,registration
127096,2021-11-29 12:14:22,2021-01-01
65712,2021-07-12 23:39:15,2021-01-01
33304,2021-04-26 12:03:07,2021-01-01


In [39]:
page_view["registration"].astype(str).str.len().min()

np.int64(3)

In [40]:
user_and_first_ts = (
    page_view.groupby("userId")
    .aggregate({"ts": "min", "registration": "min"})
    .reset_index()
)
user_and_first_ts.query("userId==25")

,userId,ts,registration
23,25.0,2021-01-05 05:01:35,2021-01-01


In [41]:
(
    page_view.merge(user_and_first_ts, on=["ts", "userId"])
    .query("itemInSession == 0")[["ts", "userId"]]
    .reset_index()
    .sort_values(by="userId")
    .groupby("userId")
    .size()
    .reset_index(name="count")
    .query("count!=1")
)

,userId,count


In [42]:
page_view["page"].unique().tolist()

['NextSong',
 'Home',
 'Upgrade',
 'Logout',
 'Login',
 'Help',
 'Downgrade',
 'Settings',
 'Save Settings',
 'About',
 'Error',
 'Submit Upgrade',
 'Submit Downgrade',
 'Cancel',
 'Cancellation Confirmation']

In [43]:
page_view["method"].unique().tolist()

['PUT', 'GET']

In [44]:
page_view.columns.tolist()

['ts',
 'sessionId',
 'page',
 'auth',
 'method',
 'status',
 'level',
 'itemInSession',
 'city',
 'zip',
 'state',
 'lon',
 'lat',
 'userId',
 'lastName',
 'firstName',
 'gender',
 'registration',
 'artist',
 'song',
 'duration']

In [45]:
counts = page_view.groupby("userId").size().reset_index(name="count")
max_mask = counts["count"].idxmax()
counts.loc[max_mask]

userId       75.0
count     13149.0
Name: 73, dtype: float64

In [46]:
mask = page_view.query("userId == 75")[["ts"]].idxmin()
page_view.loc[mask]

,ts,sessionId,page,auth,method,status,level,itemInSession,city,zip,...,lon,lat,userId,lastName,firstName,gender,registration,artist,song,duration
301,2021-01-02 07:37:40,74,NextSong,Logged In,PUT,200,free,0,Saint George,84770,...,-113.609837,37.185679,75.0,Buck,Oscar,M,2021-01-01,Hot Live Guys,I'm A Killer,138.13506


### Status Change Events Table

In [47]:
import pandas as pd

status_change = pd.read_csv(
    "/home/lupusruber/music_analytics/data/status_change_events.csv"
)

status_change.rename({"ts": "timestamp"}, axis=1, inplace=True)

status_change["timestamp"] = pd.to_datetime(status_change["timestamp"], unit="ms")
status_change["registration"] = pd.to_datetime(status_change["registration"], unit="ms")

status_change["registration"].max()

Timestamp('2021-01-01 00:00:00')

In [48]:
status_change.query("userId==25")

,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration


In [49]:
status_change["registration"].astype(str).str.len().min(), status_change[
    "registration"
].astype(str).str.len().max()

(np.int64(10), np.int64(10))

In [50]:
pd.to_datetime(1609459200000, unit="ms")

Timestamp('2021-01-01 00:00:00')

In [51]:
status_change.groupby("userId").aggregate({"timestamp": "min", "registration": "min"})

,timestamp,registration
userId,,
3,2021-03-23 15:06:09,2021-01-01
10,2021-01-08 17:03:07,2021-01-01
13,2021-06-03 07:09:13,2021-01-01
20,2021-03-16 09:25:21,2021-01-01
21,2021-09-05 16:14:19,2021-01-01
26,2021-08-11 16:18:13,2021-01-01
27,2021-01-27 07:13:06,2021-01-01
28,2021-08-06 16:32:14,2021-01-01
30,2021-01-24 11:09:03,2021-01-01


In [52]:
status_change["auth"].unique().tolist()

['Logged In']

In [53]:
status_change.columns.tolist()

['timestamp',
 'sessionId',
 'auth',
 'level',
 'itemInSession',
 'city',
 'zip',
 'state',
 'userAgent',
 'lon',
 'lat',
 'userId',
 'lastName',
 'firstName',
 'gender',
 'registration']

In [54]:
mask = status_change.query("userId == 75")[["timestamp"]].idxmin()
status_change.loc[mask]

,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
8,2021-01-29 05:33:37,1008,Logged In,free,3,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


### Listen Events Table

In [55]:
import pandas as pd

listen_events = pd.read_csv(
    r"/home/lupusruber/music_analytics/data/listen_events.csv", encoding="utf-8"
)
listen_events.rename({"ts": "timestamp"}, axis=1, inplace=True)
listen_events["registration"] = pd.to_datetime(listen_events["registration"], unit="ms")
listen_events["timestamp"] = pd.to_datetime(listen_events["timestamp"], unit="ms")

In [56]:
pd.to_datetime(listen_events["registration"], origin="unix", unit="ms").min()

Timestamp('2021-01-01 00:00:00')

In [57]:
listen_events.query("userId==75").head(3)

,artist,song,duration,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
276,Hot Live Guys,I'm A Killer,138.13506,2021-01-02 07:37:40,74,Logged In,free,0,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01
319,A-Mei,Intro,71.88853,2021-01-02 20:52:55,212,Logged In,free,0,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01
321,Amorphis,And I Hear You Call,280.97260,2021-01-02 20:54:06,212,Logged In,free,1,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


In [58]:
listen_events.columns.tolist()

['artist',
 'song',
 'duration',
 'timestamp',
 'sessionId',
 'auth',
 'level',
 'itemInSession',
 'city',
 'zip',
 'state',
 'userAgent',
 'lon',
 'lat',
 'userId',
 'lastName',
 'firstName',
 'gender',
 'registration']

In [59]:
# assert listen_events.query("timestamp <= registration").shape[0] == 0, "Invalid Rows"

In [60]:
listen_events["auth"].unique().tolist()

['Logged In']

In [61]:
mask = listen_events.query("userId == 75")[["timestamp"]].idxmin()
listen_events.loc[mask]

,artist,song,duration,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
276,Hot Live Guys,I'm A Killer,138.13506,2021-01-02 07:37:40,74,Logged In,free,0,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


In [62]:
!ls -l

total 156
-rw-r--r-- 1 lupusruber lupusruber   1308 Sep 19 11:38 spark.ipynb
-rw-r--r-- 1 lupusruber lupusruber 153907 Sep 19 13:17 tests.ipynb


### Other

In [63]:
mask = listen_events.query("userId == 75")[["timestamp"]].idxmin()
listen_events.loc[mask]

,artist,song,duration,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
276,Hot Live Guys,I'm A Killer,138.13506,2021-01-02 07:37:40,74,Logged In,free,0,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


In [64]:
mask = status_change.query("userId == 75")[["timestamp"]].idxmin()
status_change.loc[mask]

,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
8,2021-01-29 05:33:37,1008,Logged In,free,3,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


In [65]:
mask = page_view.query("userId == 75 and itemInSession==5")[["ts"]].idxmin()
page_view.loc[mask]

,ts,sessionId,page,auth,method,status,level,itemInSession,city,zip,...,lon,lat,userId,lastName,firstName,gender,registration,artist,song,duration
3980,2021-01-17 18:55:04,651,NextSong,Logged In,PUT,200,free,5,Saint George,84770,...,-113.609837,37.185679,75.0,Buck,Oscar,M,2021-01-01,Fuzz,Big Fella,168.85506


In [66]:
mask = auth_events.query("userId == 75")[["ts"]].idxmin()
auth_events.loc[mask]

,ts,sessionId,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration,success
60,2021-01-23 12:42:01,814,free,5,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75.0,Buck,Oscar,M,2021-01-01,True


In [68]:
print(auth_events["userId"].unique().shape[0])
print(page_view["userId"].unique().shape[0])
print(listen_events["userId"].unique().shape[0])
print(status_change["userId"].unique().shape[0])

90
101
100
35


In [69]:
set(page_view["userId"].unique()).difference(set(listen_events["userId"].unique()))

{np.float64(nan)}